<a href="https://colab.research.google.com/github/malinphy/Best-README-Template/blob/master/BERT_MLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#### This notebook was created to test Bert Masked Encoder to be used for Masked language model and the Seq4Rec using MultiHeadAttention Layer
# 80% Replace with a [MASK] token: For 80% of the selected inputs the token is replaced with a [MASK] token similar to the classic cloze tests mentioned earlier.
# 10% Replace with an incorrect word: For 10% of the selected inputs, the token is replaced by another randomly selected word whose only requirement is that it’s different from the selected token.
# 10% Replace with the correct word: The remaining 10% of the time the selected token is simply replaced with the correct token.



In [ ]:
### PARAMETER_LIST 
OUTPUT_LEN =256
SEQUENCE_LEN =  256
MAX_LEN = SEQUENCE_LEN
EMBED_DIM = 128
VOCAB_SIZE = 30000
TOKEN_NUM = 30000
N_NEURONS = 128
NUM_HEADS = 8
KEY_DIM = 128
NUM_ATT_LAYER = 1
SPECIAL_TOKENS=["[MASK]"]

In [ ]:
data_url = 'https://raw.githubusercontent.com/malinphy/datasets/main/IMDB_sent/IMDB%20Dataset.csv'
df = pd.read_csv(data_url)

In [ ]:
def custom_standardization(input_data):
    lowercase = str(input_data).lower()
    s = re.sub("<br />", " ",lowercase)
    s = re.sub('\x96|\x85|\xe3',' ',s) 
    out = re.sub('[%s]' % re.escape(string.punctuation), '', s)
    return out

df['review'] = df['review'].apply(custom_standardization)

In [ ]:
df.head(3)

In [ ]:
vectorize_layer = TextVectorization(
        max_tokens=VOCAB_SIZE,
        output_mode="int",
        # standardize=custom_standardization,
        output_sequence_length=SEQUENCE_LEN,
    )
vectorize_layer.adapt((df['review']))

In [ ]:
vocab = vectorize_layer.get_vocabulary()
vocab = vocab[2 : VOCAB_SIZE - len(SPECIAL_TOKENS)] + ["[mask]"]
vectorize_layer.set_vocabulary(vocab)
mask_token_id = vectorize_layer(["[mask]"]).numpy()[0][0]

In [ ]:
encoded_texts = vectorize_layer(df['review'])

In [ ]:
def get_masked_input_and_labels(encoded_texts):
    # 15% BERT masking
    inp_mask = np.random.rand(*encoded_texts.shape) < 0.15
    # Do not mask special tokens
    inp_mask[encoded_texts <= 2] = False
    # Set targets to -1 by default, it means ignore
    labels = -1 * np.ones(encoded_texts.shape, dtype=int)
    # Set labels for masked tokens
    labels[inp_mask] = encoded_texts[inp_mask]

    # Prepare input
    encoded_texts_masked = np.copy(encoded_texts)
    # Set input to [MASK] which is the last token for the 90% of tokens
    # This means leaving 10% unchanged
    inp_mask_2mask = inp_mask & (np.random.rand(*encoded_texts.shape) < 0.90)
    encoded_texts_masked[
        inp_mask_2mask
    ] = mask_token_id  # mask token is the last in the dict

    # Set 10% to a random token
    inp_mask_2random = inp_mask_2mask & (np.random.rand(*encoded_texts.shape) < 1 / 9)
    encoded_texts_masked[inp_mask_2random] = np.random.randint(
        3, mask_token_id, inp_mask_2random.sum()
    )

    # Prepare sample_weights to pass to .fit() method
    sample_weights = np.ones(labels.shape)
    sample_weights[labels == -1] = 0

    # y_labels would be same as encoded_texts i.e input tokens
    y_labels = np.copy(encoded_texts)

    return encoded_texts_masked, y_labels, sample_weights

In [ ]:
bir, iki, uc =get_masked_input_and_labels(encoded_texts)

In [ ]:
def get_pos_encoding_matrix(max_len, d_emb):
    pos_enc = np.array(
        [
            [pos / np.power(10000, 2 * (j // 2) / d_emb) for j in range(d_emb)]
            if pos != 0
            else np.zeros(d_emb)
            for pos in range(max_len)
        ]
    )
    pos_enc[1:, 0::2] = np.sin(pos_enc[1:, 0::2])  # dim 2i
    pos_enc[1:, 1::2] = np.cos(pos_enc[1:, 1::2])  # dim 2i+1
    return pos_enc

In [ ]:
first_input = tf.keras.Input(shape = (SEQUENCE_LEN,))
embedding_layer = Embedding(VOCAB_SIZE,EMBED_DIM)(first_input)
position_embeddings = layers.Embedding(
        input_dim=SEQUENCE_LEN,
        output_dim=EMBED_DIM,
        weights=[get_pos_encoding_matrix(SEQUENCE_LEN, EMBED_DIM)],
        name="position_embedding",
    )(tf.range(start=0, limit=SEQUENCE_LEN, delta=1))
# total_embs = word_embeddings + get_pos_encoding_matrix(SEQUENCE_LEN,EMBED_DIM)
# total_embs = word_embeddings + position_embeddings
x = embedding_layer + position_embeddings
for i in range(NUM_ATT_LAYER):
  mha = MultiHeadAttention(num_heads = NUM_HEADS, key_dim = 16)(x,x,x)
  norm_1 = LayerNormalization(epsilon=1e-6)(mha+x)

  seq_model = tf.keras.Sequential([
                                   Dense(N_NEURONS, activation = 'relu'),
                                   Dense(EMBED_DIM)
  ])

  seq_layer = seq_model(norm_1)
  norm_2 =LayerNormalization(epsilon=1e-6)(seq_layer+norm_1)
  x = norm_2

top_layer = Dense(VOCAB_SIZE+1, activation='relu', name = 'selection_layer')(x)
attention_model = tf.keras.Model(inputs = first_input, outputs = top_layer)
tf.keras.utils.plot_model(
    attention_model,
    to_file='model.png', show_shapes=False, show_dtype=False,
    show_layer_names=True, rankdir='TB', expand_nested=False, dpi=96,
    layer_range=None, show_layer_activations=False
)

attention_model.compile(
    loss = 'SparseCategoricalCrossentropy',
    optimizer = tf.keras.optimizers.Adam(
    learning_rate=0.001,
    #  beta_1=0.9, beta_2=0.999, epsilon=1e-07
     ),
    metrics = ['accuracy']

)

attention_model.fit(bir,iki, epochs = 12, batch_size = 30)
id2token = dict(enumerate(vectorize_layer.get_vocabulary()))
token2id = {y: x for x, y in id2token.items()}
sample_tokens = vectorize_layer(["I have watched this [mask] and it was awesome"])
prediction = attention_model.predict(sample_tokens)
mask_token_positions = np.where(sample_tokens == mask_token_id)
mask_token_position = mask_token_positions[0]
mask_prediction = prediction[0][mask_token_position]
token = np.argmax(mask_prediction)
id2token[token]